In [36]:
import os
from youtube_transcript_api import YouTubeTranscriptApi
from openai import OpenAI
from urllib.parse import urlparse, parse_qs
from dotenv import load_dotenv

In [37]:
from dotenv import load_dotenv
load_dotenv()

True

In [38]:
openai = OpenAI()

In [39]:
def get_video_id(youtube_url):
    query = urlparse(youtube_url)
    if query.hostname == 'youtu.be':
        return query.path[1:]
    if query.hostname in ('www.youtube.com', 'youtube.com'):
        if query.path == '/watch':
            return parse_qs(query.query)['v'][0]
        if query.path[:7] == '/embed/':
            return query.path.split('/')[2]
        if query.path[:3] == '/v/':
            return query.path.split('/')[2]
    raise ValueError("Invalid YouTube URL")

In [40]:
def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return " ".join([entry['text'] for entry in transcript])
    except Exception as e:
        print(f"Could not get transcript: {e}")
        return None

In [41]:
import re

def chunk_transcript(text, max_words=2000):
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    chunk = []
    word_count = 0

    for sentence in sentences:
        words = sentence.split()
        if word_count + len(words) > max_words:
            chunks.append(" ".join(chunk))
            chunk = []
            word_count = 0
        chunk.extend(words)
        word_count += len(words)

    if chunk:
        chunks.append(" ".join(chunk))

    return chunks


In [42]:
system_prompt = (
    "You are an expert tutor. "
    "Your job is to analyze the following transcript and provide a detailed, "
    "clear, and structured summary with key points, examples, and explanations. "
    "Use Markdown formatting with headings, bullet points, and bold text for clarity. "
    "If possible, break down complex ideas into simple terms."
)

In [43]:
def summarize_with_openai(text, model="gpt-4o-mini"):
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Please read this transcript and create a detailed study guide:\n\n{text}"}
        ]
    )
    return response.choices[0].message.content

In [44]:
def get_video_id(yt_url):
    # Simple extractor for ?v= or youtu.be style
    import re
    match = re.search(r"(?:v=|youtu\.be/)([a-zA-Z0-9_-]{11})", yt_url)
    if match:
        return match.group(1)
    else:
        raise ValueError("Invalid YouTube URL")


In [45]:
if __name__ == "__main__":
    yt_url = input("Enter YouTube URL: ")
    vid = get_video_id(yt_url)
    print("Video ID:", vid)

    transcript = get_transcript(vid)
    print("Transcript length:", len(transcript))

    chunks = chunk_transcript(transcript, max_words=2000)
    print(f"Total chunks: {len(chunks)}")

    summaries = []
    for i, chunk in enumerate(chunks):
        print(f"Summarizing chunk {i+1}/{len(chunks)}...")
        summary = summarize_with_openai(chunk)
        summaries.append(summary)

    combined_summary = "\n\n".join(summaries)

    print("\n=== COMBINED SUMMARY ===\n")
    print(combined_summary)

    # ✅ Final pass to polish (optional)
    final_study_guide = summarize_with_openai(
        combined_summary,
        model="gpt-4o-mini"
    )

    print("\n=== FINAL STUDY GUIDE ===\n")
    print(final_study_guide)

Video ID: K5KVEU3aaeQ
Transcript length: 96096
Total chunks: 5
Summarizing chunk 1/5...
Summarizing chunk 2/5...
Summarizing chunk 3/5...
Summarizing chunk 4/5...
Summarizing chunk 5/5...

=== COMBINED SUMMARY ===

# Python Mastery Course Study Guide

## Course Overview
This study guide outlines the key points and structure of the **Complete Python Mastery Course** designed for beginners. By the end of this course, learners will have a strong foundation in Python and be capable of utilizing it for various applications including AI, machine learning, web development, and automation.

---

## Course Introduction
- **Instructor**: M. Hamadani, a software engineer with over 20 years of experience.
- **Course Focus**: 
  - Basics to advanced concepts in Python.
  - Practical, step-by-step teaching methodology.
  - Ideal for beginners with no prior programming experience.

---

## Why Learn Python?
### Popularity and Growth
- **Fastest Growing Language**: Python is favored by software develo